In [1]:
pip install torchtune

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 350.6/350.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.7/73.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━

In [5]:
import torch
from torchtune.modules.peft import LoRALinear

torch.set_default_device("cuda")
qlora_linear = LoRALinear(512,512,rank=8,alpha=0.1,quantize_base=True)
print(torch.cuda.memory_allocated())
del qlora_linear
torch.cuda.empty_cache()

lora_linear = LoRALinear(512,512,rank=8,alpha=0.1,quantize_base=False)
print(torch.cuda.memory_allocated())
del lora_linear

169472
1081344


Using qlora in torchtune

In [2]:
#initialize qlora enabled gemma model
from torchtune.models.gemma import qlora_gemma_2b

#will apply lora to q_proj and v_proj matrics in all attn layers
#quantize these to NF4 dtype

qlora_model = qlora_gemma_2b(lora_attn_modules=["q_proj","v_proj"])


In [3]:
qlora_model

GemmaTransformerDecoder(
  (tok_embeddings): Embedding(256000, 2048)
  (layers): ModuleList(
    (0-17): 18 x TransformerDecoderLayer(
      (sa_norm): GemmaRMSNorm()
      (attn): CausalSelfAttention(
        (q_proj): LoRALinear(
          (dropout): Dropout(p=0.05, inplace=False)
          (lora_a): Linear(in_features=2048, out_features=8, bias=False)
          (lora_b): Linear(in_features=8, out_features=2048, bias=False)
        )
        (k_proj): Linear(in_features=2048, out_features=256, bias=False)
        (v_proj): LoRALinear(
          (dropout): Dropout(p=0.05, inplace=False)
          (lora_a): Linear(in_features=2048, out_features=8, bias=False)
          (lora_b): Linear(in_features=8, out_features=256, bias=False)
        )
        (output_proj): Linear(in_features=2048, out_features=2048, bias=False)
        (pos_embeddings): RotaryPositionalEmbeddings()
      )
      (mlp_norm): GemmaRMSNorm()
      (mlp): FeedForward(
        (w1): Linear(in_features=2048, out_featur

In [10]:
#k_proj and output_proj > lora not applied > parameters not quantized
attn = qlora_model.layers[0].attn
print(type(attn.q_proj.weight))
print(type(attn.k_proj.weight))

<class 'torchao.dtypes.nf4tensor.NF4Tensor'>
<class 'torch.nn.parameter.Parameter'>


In [ ]:
#need to convert nf4 back to original precision(fp32,bf16)